In [ ]:
# Install dependecies
%pip install pillow numpy opencv-python scikit-learn websockets flask

In [2]:
import os
import sys
import numpy as np

# Import external library dependencies
module_paths = [
    os.path.abspath(os.path.join('./dependencies/gaussian-splatting')),
]
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

### Running websocket proxy server:
```
conda activate SuperSplatDemo
cd .\examples\data_streaming
python .\dataserver.py
```

### Join websocket in SuperSplat web:
`Click Scene -> Start Data Server`

In [4]:
from annotatedmodel import AnnotatedModel, DatasetParams

# backCover
demo_dataset_params = DatasetParams(".\\demo\\fruit_drops_minimal.ply",
                                         ".\\demo\\fruit_drops_minimal_out.ply",
                                         ".\\demo\\fruit_drops_minimal.json",
                                         ".\\demo\\fruit_drops_minimal_out.json"
                                         )

demo = AnnotatedModel(demo_dataset_params.sh_degree)
demo.load_ply(demo_dataset_params.ply_source)

# Load existing labels with format: https://docs.segments.ai/reference/label-types#segmentation-label 
demo.load_labels(demo_dataset_params.label_source)

# Initialize new labels
#demo.initialize_empty_labels()

In [5]:
# Reupload model
def reupload_model(model:AnnotatedModel):
    model.upload_ply()
    model.upload_labels()

reupload_model(demo)

In [7]:
# K-Means
from sklearn.cluster import KMeans

def kmeans(model:AnnotatedModel, k=5):
    X = model._xyz.cpu().detach()
    clustering = KMeans(n_clusters=k).fit(X)
    labels = clustering.labels_
    print(labels)
    model.load_np_labels(labels)
    model.labelData.generate_n_categories(k-1)
    model.upload_labels()

kmeans(demo, 5)

[3 0 0 ... 2 2 1]


In [8]:
# DBSCAN
from sklearn.cluster import DBSCAN

def dbscan(model:AnnotatedModel, eps=0.02, min_samples=5):
    X = model._xyz.cpu().detach()
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    labels = clustering.labels_
    # DBSCAN uses -1 for non-clustered points
    labels = labels + 1
    k = len(np.unique(labels))
    print(f"Found K: {k}")
    print(labels)
    model.load_np_labels(labels)
    model.labelData.generate_n_categories(k)
    model.upload_labels()

dbscan(demo, 0.015, 5)

Found K: 9
[0 0 0 ... 0 0 0]


In [11]:
# Export labels in new annotations file
demo.save_labels(demo_dataset_params.label_dest)